In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log, pi, sqrt, exp
from scipy.stats import multivariate_normal
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
#import pydotplus -> baixar esse cara
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import GridSearchCV

np.random.seed(12345)

In [46]:
def fit(x, y):
    classes, ocorrencs = np.unique(y, return_counts=True) # pegando as classes e ocorrencias
    numClasses = len(classes) # numero de classes
    n = len(y) # numero de linhas do dataset
    numFeatures = x.shape[1] # numero de colunas do dataset
    
    # Probabilidade das classes será proporcional a frequencia dessa classe no dataset
    probabilidadeClasses = dict(zip(classes, ocorrencs))
    for key in probabilidadeClasses:
        probabilidadeClasses[key] = probabilidadeClasses[key] / n
    
    media = np.zeros((numFeatures, numClasses)) # criando a lista da media das features por classe
    covar = np.zeros((numFeatures, numFeatures, numClasses)) # criando a lista de matrizes de correlação das features para cada classe

    for classe in classes:
        xk = x[y == classe] # pega as linhas em que classe é igual ao y -> que aí eu acesso essas linhas do x
        classe = int(classe)
        media[:, int(classe)] = np.mean(xk, axis=0)
        xi_mean = xk - media[:, int(classe)]
        covar[:, :, int(classe)] = (np.transpose(xi_mean) @ xi_mean)/len(xk)
        covar[:, :, int(classe)] += np.eye(numFeatures) * np.mean(np.diag(covar[:,:,classe]))  * 10 ** -6 # ver se assim ta certo, pq o do cesar ele botou uns numeros que eu nao entendi
    return {'media': media, 'covar': covar, 'classes': classes, 'numRows': n, 'numClasses': numClasses, 'numFeatures': numFeatures, 'probabilidadeClasses': probabilidadeClasses }
        
def predict(model, row):
    probabilits = np.zeros(model['numClasses'])
    for classe in model['classes']:
        classe = int(classe)
        fator1 = 1/(sqrt(np.linalg.det(model['covar'][:, :, classe])) * ((2*pi)**(model['numFeatures']/2)))
        
        inversa = np.linalg.inv(model['covar'][:, :, classe])
        difXMedia = row - model['media'][:, classe]
        z = (-0.5) * (np.transpose(difXMedia) @ inversa @ difXMedia) # valor que fica dentro do exp
        probabilits[classe] = fator1 * exp(z)
    return model['classes'][np.argmax(probabilits)]
    
    
# Função para um vetor de testes e então chama o predict
def getPredict(model, x_test):
    predicted = np.array([predict(model, row) for row in x_test])
    return predicted
    

In [47]:
data = pd.read_csv("ex1.csv", header = None).values
lista = [[1,1],[1,0],[0,1],[2,4],[0.5,3],[0,0]]
X = data[:,0:-1]
Y = data[:,-1]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

In [51]:
model = fit(x_train, y_train)
results = getPredict(model, x_test)
print(y_test)

[1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]


In [52]:
tot = [1 for x,z in zip(y_test,results) if x == z]
(np.sum(np.array(tot))/len(y_test))

0.88

In [ ]:
import numpy as np
a = [[1,1,4], [2,2,3]]
uniques, countOcorrenc = np.unique(a, return_counts=True)
probs = dict(zip(uniques, countOcorrenc))
print(probs)

n = len(a)
for key in probs:
    probs[key] = probs[key] / n
print(probs)

np.mean(a, axis=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# kf = KFold(n_splits = 5, shuffle = True)
# ac = [];
# x = [1,5,10]
# #for i in x:
#     # Na linha abaixo, os parâmetros que passei para meu classificador foram, respectivamente:
#     # n_estimators (número de árvores), critério para avaliar a qualidade do split, máximo de colunas consideradas
#     # em cada split (raiz quadrada do número de colunas, que dá aprox. 7 features), profund. máx das árvores (10)
#     clf = RandomForestClassifier(i,"gini",7,10)
#     for result in kf.split(df):
#         train = df.iloc[result[0]]
#         test =  df.iloc[result[1]]
#         target_train = train[61]
#         target_test = test[61]
#         train = train.iloc[:, :59]
#         test = test.iloc[:, :59]
#         clf.fit(train, target_train)
#         target_pred = clf.predict(test)
#         ac.append(accuracy_score(target_test, target_pred))
#     print("Árvores: %d" %i)
#     print("Scores: ")
#     print(ac)
#     print("Acurácia: %f " %(mean(ac)))
#     print("\n")
#     ac = []



num_arvores = 10
max_columns = 3
max_profundidade = 10
clf = RandomForestClassifier(num_arvores,"gini",max_columns,max_profundidade)
clf.fit(train, target_train)
clf.predict(test)